In [58]:
import numpy as np
import pandas as pd
import csv
import re
import time

In [59]:
rawdata = pd.read_csv('raw_data/training_set_no_time.csv', encoding = "ISO-8859-1")

In [60]:
rawdata.head()

,Engagements,Followers at Posting,Type,Description
0,502093,36984682,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,Video,@giannis_an34 goes baseline early to rock the ...


In [61]:
rawdata['len_text'] = list(map(lambda x: len(str(x)), rawdata['Description']))

In [62]:
rawdata['word_ct'] = list(map(lambda x: str(x).count(' '), rawdata['Description']))

In [63]:
rawdata['excl_ct'] = list(map(lambda x: str(x).count('!'), rawdata['Description']))

In [64]:
rawdata['emoji_ct'] = list(map(lambda x: str(x).count('??'), rawdata['Description']))

In [65]:
rawdata['ques_ct'] = list(map(lambda x: str(x).replace("??", "").count('?'), rawdata['Description']))

In [66]:
rawdata['period_ct'] = list(map(lambda x: str(x).count('.'), rawdata['Description']))

In [67]:
rawdata['comma_ct'] = list(map(lambda x: str(x).count(','), rawdata['Description']))

In [68]:
rawdata['caps_ct'] = list(map(lambda x: len(re.findall(r'[A-Z]', str(x))), rawdata['Description']))

In [69]:
rawdata['tags_ct'] = list(map(lambda x: str(x).count('@'), rawdata['Description']))

In [70]:
import re
def hasTag(y):
    if (y is not None):
        if len(y) > 0:
            if (y[0] == '@'):
                return True
    else:
        return False

In [71]:
raw_players = list(map(lambda x: list(filter(lambda y: hasTag(y), str(x).split())), rawdata['Description']))

In [72]:
raw_players

[['@raptors', '@sergeibaka', '@normanpowell4', '@fredvanvleet'],
 ['@kyle_lowry7', '@raptors'],
 ['@k_mid22', '@bucks'],
 [],
 ['@giannis_an34'],
 ['@lukadoncic', '@traeyoung!'],
 ['@warriors'],
 ['@bucks', '@raptors'],
 ['@warriors', '@nbastore!'],
 ['@stephencurry30...', '@nbahistory!'],
 ['@stephencurry30', '@money23green'],
 ['@warriors', '@trailblazers'],
 ['@money23green', '@warriors!'],
 ['@warriors'],
 ['@money23green', '@warriors'],
 ['@stephencurry30', '@money23green', '@nbahistory'],
 ['@meyersleonard11!'],
 ['@money23green', '@warriors'],
 ['@stephencurry30'],
 ['@stephencurry30', '@warriors'],
 ['@damianlillard', '@trailblazers'],
 ['@meyersleonard11', '@trailblazers'],
 ['@stephencurry30'],
 ['@money23green'],
 ['@budweiserusa', '@trailblazers', '@trailblazers', '@warriors?'],
 ['@money23green', '@warriors', '@trailblazers', '@nbaonespn'],
 ['@trailblazers...', '@warriors', '@nbaonespn!'],
 ['@raptors'],
 ['@raptors', '@kristenledlow!'],
 ['@raptors'],
 ['@raptors'],
 ['@

In [73]:
##regex = ' \s+\n,.-_!?/\\><()&'
regex = "[\s\n\x91\x92,-/\\><()&!?\*^#|:;']+"
rawdata['player'] = list(map(lambda x: list(filter(lambda y: hasTag(y), re.split(regex, str(x)))), rawdata['Description']))

In [74]:
rawdata['player'].head()

0    [@raptors, @sergeibaka, @normanpowell4, @fredv...
1                             [@kyle_lowry7, @raptors]
2                                   [@k_mid22, @bucks]
3                                                   []
4                                      [@giannis_an34]
Name: player, dtype: object

In [75]:
teams = pd.read_csv('raw_data/teams.csv')
teams.head()

,Team Name,Insta_ID,Number of Posts,Number of Followers
0,Atlanta Hawks,atlhawks,7336,1M
1,Boston Celtics,celtics,7822,4.2M
2,Brooklyn Nets,brooklynnets,6363,1.2M
3,Charlotte Hornets,hornets,9653,1M
4,Chicago Bulls,chicagobulls,5202,4.4M


In [76]:
# function for pulling instagram followers:
import urllib.request, json 
def get_followers(searchquery):
    url = 'https://www.instagram.com/' + searchquery + '/?__a=1'
    with urllib.request.urlopen(url) as u:
        data = json.loads(u.read().decode())
        return data['graphql']['user']['edge_followed_by']['count']

In [77]:
# distinguish player from team:
def get_player_followers(searchquery):
    print(searchquery)
    time.sleep(2)
    if searchquery not in teams['Insta_ID']:
        followers = 0
        try:
            followers = get_followers(searchquery)
        except Exception:
            print("HAS ERROR")
        return followers
    return 0
# team:
def get_team_followers(searchquery):
    print(searchquery)
    time.sleep(2)
    if searchquery in teams['Insta_ID']:
        followers = 0
        try:
            followers = get_followers(searchquery)
        except Exception:
            print("HAS ERROR")
        return followers
    return 0

In [78]:
## Gather the follower count for all players:
player_list = []
rawdata['player'].values
for i in rawdata['player']:
    player_list = player_list + i
player_set = list(set(player_list))
print(len(player_set))
print(len(player_list))
## Gather count for all teams:
team_set = teams['Insta_ID']
print(len(team_set))

848
11714
30


In [79]:
# player fc
pfc = pd.DataFrame(player_set)
pfc.columns = ['player']
# team fc
tfc = pd.DataFrame(team_set)
tfc.columns = ['team']

In [80]:
pfc['fc'] = pfc['player'].apply(lambda x: get_player_followers(x.strip('@')))


bronzewhale
mb3five
danilogallogallinari
statefarm
realtristan13
jrue_holiday11
realjaywilliams
alexlen_21
ygtrece
Raptors905
swipathefox
JHarden13
roadtoriches_0
MGMResortsIntl
troywilliams_
bosnianbeast27
iamjamiefoxx
kingjames
masfresco
iam_j3
capitalcitygogo
thebledshow
stephensonlance
meyersleonard11
khembirch
lsubasketball
treydaviis
jjwatt
jasonwilliams55
pr00fessortrill
bam1of1
dlamarc33
furkankorkkmaz
kumhotireusa
thanasis_ante43
skal_lab
MiamiHEAT
Spurs_Austin
officialzobrown
javalemcgee
ianou76
dulenader2
juanchiviris41
laclippers
SacramentoKings
bobimarjanovic13
pharrell
wayneseldenjr
carmeloanthony
tpinsonn
NBASummerLeague
kobebryant
BudweiserUSA
northcoastbluechips
ipjh55
lindsaywhalen
chelseafc
tomasatoransky
supermariohezonja
fchwpo
okarowhite
americanairlinesarena
andredrummondd
jimmybutler
nbastore
fergski23
MooreMaya
common
taureanprince
3tross1
normanpowell4
josh_j11
dbacon
joeharris__12
spurs_austin
espn
the_4th_holiday
jmcalderon8
quavohuncho
dickssportinggoods
th

pjtucker
rickandmorty
afterschoolallstars
j3vans1
ds17_fg
iamjustise
stjude
original00g
sdotcurry
utahjazzgaming
burniemiamiheat
candaceparker
bucks
BreannaStewart30
instagram
trezz24
omarispellman
darrencollison_
btyphoto
dameology
sportscenter
BrooklynNets
briskuno
thonmaker
djkhaled
wendellcarterjr
ktoli20
nbaacademy
dallasmavs
gustogummi88
youngamechanger
sloot22
maximilian
lukadoncic
mentornmp
kostas__ante13
arsenal
kevinlove
milesbridges
karltowns
fearthefro95
35_fitz
knicksgaming
jabariparker
thats_g_
beatsbydre
dzananmusa
beyonce
sacramentokings
officialspikelee
vicoladipo
darrenrovell
nerlensnoel3
budweiserusa
OKC
tmac213
brookiethewookie11
HAS ERROR
bogdanbogdanovic
chiefhasarrived
KobeBryant
nicbatum88
the2kferguson
teamslowmo
antonio_michail
irie
mrjloyd
delonwright
jah8
mache275
djjasonleroux
yutawatanabe12
nuggets
ivicazubac
garydwayne
alexabrines
t
HAS ERROR
moe_harkless
brittneyyevettegriner
la_sparks
chadwickboseman
guerschony
swish41
dequanmjones
trayfour
joemurphypho

In [81]:
tfc['fc'] = tfc['team'].apply(lambda x: get_team_followers(x.strip('@')))


atlhawks
celtics
brooklynnets
hornets
chicagobulls
cavs
dallasmavs
nuggets
detroitpistons
warriors
houstonrockets
pacers
laclippers
lakers
memgrizz
miamiheat
bucks
timberwolves
pelicansnba
nyknicks
okcthunder
orlandomagic
sixers
suns
trailblazers
sacramentokings
spurs
raptors
utahjazz
washwizards


In [82]:
pfc.to_csv('player_follower_count.csv')


In [83]:
pfc = pd.read_csv('player_follower_count.csv', index_col=1)

In [94]:
import operator
import functools
def follower_count_text():
    # 1 - (1-pA1)(1-pA2)...
#     print(pfc[:5])
    tmp_list = [[pfc.loc[player, 'fc'] if player in pfc.index else 0 for player in list_players] for list_players in raw_players]
    tot_fc = []
    for list_players in tmp_list:
        total = sum(list_players)
        if total == 0:
            tot_fc.append(0)
            continue
        newVal = (1 - functools.reduce(operator.mul, [1-v/total for v in list_players], 1)) * total
        tot_fc.append(newVal)
    return(tot_fc)

follower_count_text()
len(follower_count_text())

7766

## Merge follower count w/ main data

## Train RF and assess w/ CV

## Train and assess keras NN (https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

In [49]:
import matplotlib.pyplot as plt
plt.scatter(rawdata['word_ct'], rawdata['len_text'])
plt.show()

<Figure size 640x480 with 1 Axes>

In [24]:
teams.head()

,Team Name,Insta_ID,Number of Posts,Number of Followers
0,Atlanta Hawks,atlhawks,7336,1M
1,Boston Celtics,celtics,7822,4.2M
2,Brooklyn Nets,brooklynnets,6363,1.2M
3,Charlotte Hornets,hornets,9653,1M
4,Chicago Bulls,chicagobulls,5202,4.4M


In [21]:
from lxml import html
import requests
from lxml.html import fromstring, tostring


In [42]:
searchquery = 'stephencurry30'
url = 'https://www.instagram.com/' + searchquery + '/?__a=1'
page = requests.get(url)
tree = html.fromstring(page.content)


In [50]:
import urllib.request, json 
with urllib.request.urlopen(url) as u:
    data = json.loads(u.read().decode())
    print(data)

{'logging_page_id': 'profilePage_324599988', 'show_suggested_profiles': True, 'graphql': {'user': {'biography': '@vsonwatch', 'blocked_by_viewer': False, 'country_block': False, 'external_url': 'http://fb.me/StephenvsTheGame', 'external_url_linkshimmed': 'https://l.instagram.com/?u=http%3A%2F%2Ffb.me%2FStephenvsTheGame&e=ATO5G_MDgJUOJZ43Bq_Bb4jIO0lAw12fKgakgMOZlFULA1TTeH3FHag-vrLck7bIEgdMLV-I', 'edge_followed_by': {'count': 25661091}, 'followed_by_viewer': False, 'edge_follow': {'count': 648}, 'follows_viewer': False, 'full_name': 'Wardell Curry', 'has_channel': False, 'has_blocked_viewer': False, 'highlight_reel_count': 4, 'has_requested_viewer': False, 'id': '324599988', 'is_business_account': False, 'is_joined_recently': False, 'business_category_name': None, 'is_private': False, 'is_verified': True, 'edge_mutual_followed_by': {'count': 0, 'edges': []}, 'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/eac4ca641e352ef94e652fb39583f44a/5D859F9A/t51.2885-19/s150x150/2227

In [52]:
data['logging_page_id']

'profilePage_324599988'

In [53]:
data['graphql']

{'user': {'biography': '@vsonwatch',
  'blocked_by_viewer': False,
  'country_block': False,
  'external_url': 'http://fb.me/StephenvsTheGame',
  'external_url_linkshimmed': 'https://l.instagram.com/?u=http%3A%2F%2Ffb.me%2FStephenvsTheGame&e=ATO5G_MDgJUOJZ43Bq_Bb4jIO0lAw12fKgakgMOZlFULA1TTeH3FHag-vrLck7bIEgdMLV-I',
  'edge_followed_by': {'count': 25661091},
  'followed_by_viewer': False,
  'edge_follow': {'count': 648},
  'follows_viewer': False,
  'full_name': 'Wardell Curry',
  'has_channel': False,
  'has_blocked_viewer': False,
  'highlight_reel_count': 4,
  'has_requested_viewer': False,
  'id': '324599988',
  'is_business_account': False,
  'is_joined_recently': False,
  'business_category_name': None,
  'is_private': False,
  'is_verified': True,
  'edge_mutual_followed_by': {'count': 0, 'edges': []},
  'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/eac4ca641e352ef94e652fb39583f44a/5D859F9A/t51.2885-19/s150x150/22277378_1720913538216240_2580026733478543360_n.jpg

In [54]:
data['graphql']['user']['edge_followed_by']['count']

25661091

In [52]:
rawdata.head()

,Engagements,Followers at Posting,Type,Description,len_text,word_ct,excl_ct,ques_ct,period_ct,comma_ct,caps_ct,tags_ct,player
0,502093,36984682,Video,The @raptors bench trio of @sergeibaka @norman...,95,14,1,0,0,0,2,4,"[@raptors, @sergeibaka, @normanpowell4, @fredv..."
1,603380,36984682,Video,@kyle_lowry7 pulls from deep for the @raptors ...,64,11,1,0,0,0,3,2,"[@kyle_lowry7, @raptors]"
2,603380,36984682,Video,@k_mid22 with some english on the @bucks dime!,46,7,1,0,0,0,0,2,"[@k_mid22, @bucks]"
3,725100,36984682,Video,Kawhi punches it home with the left on TNT!,43,8,1,0,0,0,4,0,[]
4,661446,36984682,Video,@giannis_an34 goes baseline early to rock the ...,57,9,1,0,0,0,3,1,[@giannis_an34]
